**Keras Sequential/Functional API 模式建立模型**

最典型和常用的神經網路結構是將一堆層按特定順序疊加起來，那麼，我們是不是只需要提供一個層的列表，就能由 Keras 將它們自動首尾相連，形成模型呢？Keras 的 Sequential API 正是如此。通過向 tf.keras.models.Sequential() 提供一個層的列表，就能快速地建立一個 tf.keras.Model 模型並返回：

In [19]:
import tensorflow as tf
import numpy as np

class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [20]:
#1.繼承keras.Model, call()定義任意複雜的網路結構
#2.keras.Model構造函數, 傳入inputs和outputs, 定義任意複雜的網路結構
#3.keras.models.Sequential(), 只能定義簡單的1層傳1層的網路結構
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

不過，這種層疊結構並不能表示任意的神經網路結構。為此，Keras 提供了 Functional API，幫助我們建立更為複雜的模型，例如多輸入 / 輸出或存在參數共用的模型。其使用方法是將層作為可調用的對象並返回張量（這點與之前章節的使用方法一致），並將輸入向量和輸出向量提供給 tf.keras.Model 的 inputs 和 outputs 參數，範例如下：

In [21]:
#更靈活的模型結構定義方式
# inputs = tf.keras.Input(shape=(28, 28, 1))
# x = tf.keras.layers.Flatten()(inputs)
# x = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)(x)
# x = tf.keras.layers.Dense(units=10)(x)
# outputs = tf.keras.layers.Softmax()(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

**使用 Keras Model 的 compile 、 fit 和 evaluate 方法訓練和評估模型**

當模型建立完成後，通過 tf.keras.Model 的 compile 方法配置訓練過程：

In [22]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [23]:
data_loader = MNISTLoader()

model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

Epoch 1/5
1200/1200 [==============================] - 4s 3ms/step - loss: 0.3065 - sparse_categorical_accuracy: 0.9138
Epoch 2/5
1200/1200 [==============================] - 3s 3ms/step - loss: 0.1418 - sparse_categorical_accuracy: 0.9593
Epoch 3/5
1200/1200 [==============================] - 4s 4ms/step - loss: 0.1025 - sparse_categorical_accuracy: 0.9705
Epoch 4/5
1200/1200 [==============================] - 4s 4ms/step - loss: 0.0786 - sparse_categorical_accuracy: 0.9768
Epoch 5/5
1200/1200 [==============================] - 3s 2ms/step - loss: 0.0622 - sparse_categorical_accuracy: 0.9815


In [24]:
print(model.evaluate(data_loader.test_data, data_loader.test_label))

313/313 [==============================] - 0s 1ms/step - loss: 0.0919 - sparse_categorical_accuracy: 0.9727
[0.09186572581529617, 0.9726999998092651]


**自定義層、損失函數和評量指標**

可能你還會問，如果現有的這些層無法滿足我的要求，我需要定義自己的層怎麼辦？事實上，我們不僅可以繼承 tf.keras.Model 編寫自己的模型類，也可以繼承 tf.keras.layers.Layer 編寫自己的層。

**自定義層**

自定義層需要繼承 tf.keras.layers.Layer 類，並覆寫 __init__ 、 build 和 call 三個方法，如下所示：

In [25]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        # 初始化程式碼

    def build(self, input_shape):     # input_shape 是一個 TensorShape 類型對象，提供輸入的形狀
        # 在第一次使用該層的時候呼叫該部分程式碼，在這裡創建變數可以使得變數的形狀自適應輸入
        # 而不需要使用者額外指定變數形狀。
        # 如果已經可以完全確定變數的形狀，也可以在__init__部分創建變數
        self.variable_0 = self.add_weight(...)
        self.variable_1 = self.add_weight(...)

    def call(self, inputs):
        # 模型呼叫的程式碼（處理輸入並返回輸出）
        return output

例如，如果我們要自己實現一個 本章第一節 中的全連接層（ tf.keras.layers.Dense ），可以按以下方式編寫。此程式碼在 build 方法中創建兩個變數，並在 call 方法中使用創建的變數進行運算：

In [26]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

num_epochs = 5
batch_size = 50
learning_rate = 0.001

class LinearLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):     # 這裡 input_shape 是第一次運行call()時參數inputs的形狀
        self.w = self.add_variable(name='w',
            shape=[input_shape[-1], self.units], initializer=tf.random_normal_initializer())
        self.b = self.add_variable(name='b',
            shape=[self.units], initializer=tf.random_normal_initializer())

    def call(self, inputs):
        y_pred = tf.nn.relu(tf.matmul(inputs, self.w) + self.b)
        return y_pred

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    LinearLayer(100),
    LinearLayer(10),
    tf.keras.layers.Softmax()
])

在定義模型的時候，我們便可以如同 Keras 中的其他層一樣，呼叫我們自定義的層 LinearLayer：

In [27]:
class LinearModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer = LinearLayer(units=1)

    def call(self, inputs):
        output = self.layer(inputs)
        return output

**自定義損失函數和評量指標**
 
自定義損失函數需要繼承 tf.keras.losses.Loss 類別，重寫 call 方法即可，輸入真實值 y_true 和模型預測值 y_pred ，輸出模型預測值和真實值之間通過自定義的損失函數計算出的損失值。下面的範例為均方差 (Mean-Square Error, MSE）損失函數：

In [28]:
class MeanSquaredError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        y_true = tf.one_hot(tf.cast(y_true, dtype='int32'), depth=10)
        return tf.reduce_mean(tf.reduce_sum(tf.square(y_pred - y_true)))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=MeanSquaredError(),
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

data_loader = MNISTLoader()

model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

自定義評量指標需要繼承 tf.keras.metrics.Metric 類，並重寫 __init__ 、 update_state 和 result 三個方法。下面的範例對前面用到的 SparseCategoricalAccuracy 評量指標類別做了一個簡單的重實現：

In [29]:
class SparseCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self):
        super().__init__()
        self.total = self.add_weight(name='total', dtype=tf.int32, initializer=tf.zeros_initializer())
        self.count = self.add_weight(name='count', dtype=tf.int32, initializer=tf.zeros_initializer())

    def update_state(self, y_true, y_pred, sample_weight=None):
        values = tf.cast(tf.equal(y_true, tf.argmax(y_pred, axis=-1, output_type=tf.int32)), tf.int32)
        self.total.assign_add(tf.shape(y_true)[0])
        self.count.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.count / self.total